In [7]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [8]:
weak_masks_path = 'train_dataset'

In [9]:
all_files = [f for f in os.listdir(weak_masks_path) if f.endswith('.npy')]

In [10]:
len(all_files)

26394

In [5]:
def visualize_opf_vs_gt(data_folder, sample_ids=None, max_samples=5):
    print("Total files:", len(all_files))

    if sample_ids:
        npy_files = [f"{sid}.npy" for sid in sample_ids if f"{sid}.npy" in all_files]
    else:
        npy_files = all_files[:max_samples]

    for file in npy_files:
        path = os.path.join(data_folder, file)
        data = np.load(path, allow_pickle=True)

        image = data[0]                # (H, W, 3) or similar
        gt_mask = data[1][0]           # (H, W)
        opf_mask = data[5][0]          # (H, W)
        print("Sample:", file)

        print("Ground Truth mask values:", np.unique(gt_mask))
        print("OPF mask values:", np.unique(opf_mask))

        # Normalize image for display
        img_disp = np.transpose(image, (1, 2, 0))
        img_disp = (img_disp - img_disp.min()) / (img_disp.max() - img_disp.min())

        # Transpose only masks
        gt_disp = np.squeeze(gt_mask).T
        opf_disp = np.squeeze(opf_mask).T

        plt.figure(figsize=(12, 4))
        plt.suptitle(f"Sample: {file.replace('.npy', '')}", fontsize=14)

        plt.subplot(1, 3, 1)
        plt.imshow(img_disp)
        plt.title("Original Image")
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(gt_disp, cmap='gray', vmin=0, vmax=1)
        plt.title("Ground Truth")
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(opf_disp, cmap='gray', vmin=0, vmax=1)
        plt.title("OPF Mask")
        plt.axis('off')

        plt.tight_layout()
        plt.show()

In [6]:
visualize_opf_vs_gt(weak_masks_path, sample_ids=["r001s001_104"])

Total files: 26394
Sample: r001s001_104.npy
Ground Truth mask values: [0. 1.]
OPF mask values: [0. 1.]


NameError: name 'plt' is not defined

### Computing IOU between the Ground truth masks and OPF masks (both are {0.0, 1.0} arrays)

In [11]:
def compute_iou(mask1, mask2):
    """
    Compute IoU between two binary masks.
    """
    mask1 = (mask1 > 0)
    mask2 = (mask2 > 0)

    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()

    if union == 0:
        return 0.0
    return intersection / union

### Selecting top "n" representative samples based on IOU approach

In [8]:
import csv

def select_samples_by_iou(weak_masks_path, n, strategy='highest', output_csv=None):
    """
    Selects top-n samples based on IoU between OPF masks and ground truth masks.

    This function supports resuming by saving progress to a CSV file. It skips already
    processed samples and handles corrupt files safely.

    Args:
        weak_masks_path (str): Folder containing .npy files with image + mask data.
        n (int): Number of samples to select.
        strategy (str): 'highest' for best IoUs, 'lowest' for worst IoUs.
        output_csv (str): Optional path to CSV file for saving and resuming progress.

    Returns:
        List of (sample_id, iou_score) tuples sorted by IoU.
    """
    iou_scores = []
    processed_ids = set()

    if output_csv and os.path.exists(output_csv):
        with open(output_csv, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                if row and row[0] != 'sample_id':
                    processed_ids.add(row[0])
                    iou_scores.append((row[0], float(row[1])))


    for file in all_files:
        sample_id = file.replace(".npy", "")
        if sample_id in processed_ids:
            print(f"Skipping already processed: {sample_id}")
            continue

        file_path = os.path.join(weak_masks_path, file)
        try:
            data = np.load(file_path, allow_pickle=True)
            gt_mask = data[1][0]
            opf_mask = data[5][0]
            iou = compute_iou(gt_mask, opf_mask)
            iou_scores.append((sample_id, iou))
            print(f"Sample: {sample_id}, IOU: {iou:.4f}")

            if output_csv:
                os.makedirs(os.path.dirname(output_csv), exist_ok=True)
                mode = 'a' if os.path.exists(output_csv) else 'w'
                with open(output_csv, mode, newline='') as f:
                    writer = csv.writer(f)
                    if mode == 'w':
                        writer.writerow(['sample_id', 'iou_score'])
                    writer.writerow([sample_id, f"{iou:.4f}"])

        except Exception as e:
            print(f"Skipping {file} due to error: {e}")
            continue

    reverse = strategy == "highest"
    sorted_samples = sorted(iou_scores, key=lambda x: x[1], reverse=reverse)
    print(f"Found {len(all_files)} files in {weak_masks_path}")
    print(f"Already processed: {len(processed_ids)}")
    print(len(sorted_samples))
    return sorted_samples[:n]

In [10]:
selected = select_samples_by_iou(
    weak_masks_path=weak_masks_path,
    n=240,
    strategy='lowest',
    output_csv='/mnt/iusers01/fse-ugpgt01/compsci01/f32077dn/scratch/iou_scores.csv'
)

Skipping already processed: r038s010_92
Skipping already processed: r048s016_53
Skipping already processed: r034s007_87
Sample: r038s032_80, IOU: 0.4296
Sample: r031s011_68, IOU: 0.5910
Skipping already processed: r028s008_135
Skipping already processed: r038s047_116
Skipping already processed: r038s064_126
Sample: r038s060_108, IOU: 0.7381
Skipping already processed: r031s032_86
Sample: r052s029_69, IOU: 0.8333
Sample: r047s039_64, IOU: 0.2500
Sample: r009s066_126, IOU: 0.7000
Skipping already processed: r048s016_64
Skipping already processed: r010s008_71
Skipping already processed: r027s048_75
Skipping already processed: r031s035_96
Sample: r029s010_21, IOU: 0.3659
Sample: r001s005_115, IOU: 0.5191
Sample: r052s025_63, IOU: 0.2695
Sample: r009s050_59, IOU: 0.6122
Sample: r029s003_106, IOU: 0.1093
Skipping already processed: r046s007_47
Sample: r001s036_69, IOU: 0.5350
Sample: r003s003_136, IOU: 0.7500
Skipping already processed: r038s071_85
Sample: r009s092_108, IOU: 0.8333
Skipping 

Sample: r001s005_84, IOU: 0.1990
Skipping already processed: r034s015_81
Skipping already processed: r019s009_100
Sample: r042s028_27, IOU: 0.5806
Sample: r031s030_94, IOU: 0.5917
Skipping already processed: r009s121_62
Sample: r047s046_69, IOU: 0.3538
Skipping already processed: r018s012_106
Sample: r019s001_143, IOU: 0.4286
Sample: r015s008_94, IOU: 0.3487
Sample: r005s015_147, IOU: 0.6273
Sample: r048s015_70, IOU: 0.7008
Skipping already processed: r024s005_135
Sample: r040s022_89, IOU: 0.5824
Skipping already processed: r038s043_124
Sample: r001s005_116, IOU: 0.7598
Sample: r042s028_134, IOU: 0.4909
Sample: r031s011_79, IOU: 0.4307
Skipping already processed: r004s020_101
Sample: r040s032_111, IOU: 0.6471
Skipping already processed: r004s019_35
Sample: r031s015_98, IOU: 0.6632
Skipping already processed: r001s008_46
Sample: r047s021_33, IOU: 0.7795
Skipping already processed: r040s054_73
Skipping already processed: r009s055_118
Skipping already processed: r002s008_79
Skipping alrea

Sample: r034s046_35, IOU: 0.5000
Skipping already processed: r010s006_74
Skipping already processed: r038s089_88
Skipping already processed: r009s099_98
Skipping already processed: r017s116_75
Sample: r034s025_74, IOU: 0.8018
Skipping already processed: r052s032_104
Skipping already processed: r042s008_114
Sample: r003s013_94, IOU: 0.6475
Skipping already processed: r038s018_108
Sample: r031s013_58, IOU: 0.6713
Sample: r004s016_114, IOU: 0.6984
Sample: r009s083_92, IOU: 0.3544
Sample: r031s010_103, IOU: 0.4778
Skipping already processed: r024s013_71
Skipping already processed: r052s031_84
Skipping already processed: r009s067_118
Sample: r040s015_28, IOU: 0.5452
Skipping already processed: r034s039_46
Skipping already processed: r001s013_88
Skipping already processed: r001s034_68
Skipping already processed: r050s003_37
Skipping already processed: r004s025_111
Skipping already processed: r005s070_88
Skipping already processed: r028s009_72
Skipping already processed: r015s025_91
Skipping 

Sample: r001s021_96, IOU: 0.4333
Skipping already processed: r038s022_58
Sample: r048s036_64, IOU: 0.5073
Sample: r011s016_81, IOU: 0.6000
Skipping already processed: r027s047_81
Skipping already processed: r015s016_109
Skipping already processed: r038s014_76
Sample: r002s010_111, IOU: 0.2697
Sample: r047s026_86, IOU: 0.6331
Skipping already processed: r040s044_53
Skipping already processed: r028s013_65
Skipping already processed: r009s032_104
Skipping already processed: r052s031_87
Sample: r004s023_135, IOU: 0.4791
Skipping already processed: r034s039_61
Sample: r001s021_98, IOU: 0.5577
Sample: r019s001_145, IOU: 0.5556
Sample: r048s032_122, IOU: 0.6477
Skipping already processed: r004s003_98
Skipping already processed: r001s020_70
Skipping already processed: r052s031_37
Skipping already processed: r038s084_120
Sample: r009s083_78, IOU: 0.5349
Sample: r042s013_96, IOU: 0.5211
Sample: r038s091_101, IOU: 0.6333
Sample: r001s036_26, IOU: 0.2564
Skipping already processed: r002s011_93
Ski

Sample: r031s002_116, IOU: 0.2131
Sample: r009s006_74, IOU: 0.1051
Skipping already processed: r047s041_120
Skipping already processed: r017s116_102
Sample: r029s004_91, IOU: 0.3762
Sample: r011s025_82, IOU: 0.6250
Skipping already processed: r019s004_138
Skipping already processed: r034s039_76
Sample: r040s043_115, IOU: 0.6923
Sample: r042s013_86, IOU: 0.4580
Skipping already processed: r002s006_66
Skipping already processed: r038s078_75
Skipping already processed: r009s085_108
Sample: r040s069_78, IOU: 0.3584
Sample: r038s085_78, IOU: 0.4000
Skipping already processed: r011s030_76
Skipping already processed: r029s007_81
Skipping already processed: r048s016_48
Skipping already processed: r015s018_41
Skipping already processed: r034s014_110
Skipping already processed: r040s037_118
Skipping already processed: r028s014_22
Sample: r003s003_79, IOU: 0.6858
Sample: r018s011_121, IOU: 0.5222
Sample: r029s003_81, IOU: 0.5764
Skipping already processed: r009s021_80
Skipping already processed: 

...

Sample: r038s028_60, IOU: 0.5355
Sample: r009s002_68, IOU: 0.6000
Skipping already processed: r038s056_100
Skipping already processed: r004s004_82
Sample: r040s048_62, IOU: 0.5800
Sample: r027s009_93, IOU: 0.7033
Sample: r004s036_76, IOU: 0.4934
Sample: r027s015_91, IOU: 0.7538
Sample: r001s033_106, IOU: 0.6863
Skipping already processed: r001s009_133
Skipping already processed: r009s027_31
Skipping already processed: r001s009_109
Sample: r031s011_53, IOU: 0.5865
Sample: r009s083_117, IOU: 0.2591
Sample: r042s028_93, IOU: 0.6690
Skipping already processed: r003s008_89
Skipping already processed: r040s047_129
Skipping already processed: r002s011_123
Sample: r010s012_75, IOU: 0.0000
Skipping already processed: r009s085_39
Skipping already processed: r011s030_93
Skipping already processed: r001s010_30
Skipping already processed: r038s078_62
Sample: r038s026_66, IOU: 0.4615
Skipping already processed: r028s008_87
Sample: r031s008_94, IOU: 0.7045
Skipping already processed: r038s023_99
Skip

Sample: r052s002_93, IOU: 0.6408
Skipping already processed: r009s085_17
Skipping already processed: r031s019_122
Skipping already processed: r009s085_38
Sample: r031s003_136, IOU: 0.3715
Skipping already processed: r003s008_72
Sample: r004s035_89, IOU: 0.7710
Sample: r009s089_107, IOU: 0.2944
Skipping already processed: r034s013_139
Skipping already processed: r038s018_102
Skipping already processed: r004s028_71
Sample: r040s072_85, IOU: 0.3347
Skipping already processed: r034s011_78
Skipping already processed: r040s037_104
Sample: r042s028_100, IOU: 0.6551
Sample: r003s013_68, IOU: 0.7692
Skipping already processed: r040s075_88
Skipping already processed: r040s086_95
Skipping already processed: r042s025_86
Skipping already processed: r024s018_97
Sample: r038s054_96, IOU: 0.7679
Skipping already processed: r042s018_124
Skipping already processed: r009s088_30
Skipping already processed: r001s010_55
Sample: r031s018_102, IOU: 0.4222
Skipping already processed: r004s009_104
Sample: r005s

Sample: r038s054_72, IOU: 0.5815
Sample: r003s003_110, IOU: 0.3971
Skipping already processed: r038s071_43
Skipping already processed: r034s039_56
Skipping already processed: r009s055_74
Sample: r009s108_53, IOU: 0.7259
Skipping already processed: r001s013_127
Skipping already processed: r005s075_95
Skipping already processed: r009s096_99
Skipping already processed: r002s006_78
Sample: r038s028_91, IOU: 0.5259
Sample: r040s008_116, IOU: 0.5800
Sample: r018s011_120, IOU: 0.7282
Sample: r009s052_105, IOU: 0.2458
Skipping already processed: r004s020_55
Sample: r009s110_93, IOU: 1.0000
Skipping already processed: r005s074_125
Sample: r015s010_123, IOU: 0.5087
Skipping already processed: r001s034_66
Skipping already processed: r004s003_87
Skipping already processed: r042s008_97
Sample: r002s003_106, IOU: 0.5333
Skipping already processed: r001s016_92
Skipping already processed: r017s116_117
Skipping already processed: r019s010_56
Skipping already processed: r002s005_90
Sample: r034s009_108,

Sample: r010s018_68, IOU: 0.3333
Skipping already processed: r027s050_70
Sample: r003s013_63, IOU: 0.6200
Skipping already processed: r042s008_111
Sample: r003s013_71, IOU: 0.7660
Skipping already processed: r038s078_95
Skipping already processed: r038s014_69
Skipping already processed: r004s029_140
Skipping already processed: r004s030_100
Skipping already processed: r052s032_95
Sample: r040s071_135, IOU: 0.0956
Sample: r031s002_150, IOU: 0.8125
Skipping already processed: r038s048_40
Sample: r004s016_64, IOU: 0.6667
Skipping already processed: r034s047_71
Sample: r009s108_63, IOU: 0.3781
Skipping already processed: r031s031_96
Skipping already processed: r001s020_41
Skipping already processed: r002s005_96
Skipping already processed: r038s041_98
Skipping already processed: r001s012_117
Skipping already processed: r001s010_118
Sample: r040s071_134, IOU: 0.1383
Skipping already processed: r052s032_74
Skipping already processed: r040s010_84
Skipping already processed: r009s085_27
Sample: 

Sample: r031s010_84, IOU: 0.6449
Skipping already processed: r031s022_109
Sample: r010s010_99, IOU: 0.4000
Skipping already processed: r042s008_80
Skipping already processed: r034s048_85
Skipping already processed: r004s028_54
Skipping already processed: r040s075_96
Skipping already processed: r038s061_80
Skipping already processed: r004s010_82
Skipping already processed: r009s067_46
Skipping already processed: r004s029_91
Skipping already processed: r027s050_93
Skipping already processed: r002s005_81
Skipping already processed: r009s088_94
Skipping already processed: r034s032_80
Skipping already processed: r009s064_86
Skipping already processed: r040s037_130
Skipping already processed: r004s020_71
Sample: r009s010_45, IOU: 0.7421
Sample: r003s003_118, IOU: 0.5455
Skipping already processed: r004s019_31
Skipping already processed: r004s004_83
Skipping already processed: r005s031_120
Skipping already processed: r034s039_82
Sample: r003s009_79, IOU: 0.4444
Skipping already processed: r04

In [11]:
csv_path = "iou_scores.csv"
with open(csv_path, 'r') as file:
    num_records = sum(1 for line in file) - 1  # Subtract 1 for the header

print(f"Number of records in CSV: {num_records}")

Number of records in CSV: 26393


## Returns top n samples but we want to select top n mri scans.
#### But atleast we have the csv file with the IOU values of each sample, so we can just group them and find the top n mri scans using the cummulative value of samples.

In [13]:
import csv
from collections import defaultdict

def select_top_scans_from_csv_and_save(iou_csv_path, n=10, strategy='highest', output_txt_path=None):
    """
    Groups slices by scan ID and selects top-n scans based on cumulative IoU.
    Saves the result to a text file if output_txt_path is provided.

    Args:
        iou_csv_path (str): Path to CSV file with columns: sample_id,iou_score
        n (int): Number of scan IDs to return
        strategy (str): 'highest' or 'lowest'
        output_txt_path (str, optional): Path to save selected scan IDs and scores

    Returns:
        List[Tuple[str, float]]: List of (scan_id, total_iou)
    """
    scan_to_total_iou = defaultdict(float)

    with open(iou_csv_path, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if row[0] == 'sample_id':
                continue
            sample_id, iou = row[0], float(row[1])
            scan_id = sample_id.split('_')[0]
            scan_to_total_iou[scan_id] += iou

    # Sort scans
    reverse = (strategy == 'highest')
    sorted_scans = sorted(scan_to_total_iou.items(), key=lambda x: x[1], reverse=reverse)
    top_scans = sorted_scans[:n]

    # Save to file if requested
    if output_txt_path:
        os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
        with open(output_txt_path, 'w') as f:
            for scan_id, total_iou in top_scans:
                f.write(f"{scan_id},{total_iou:.4f}\n")
        print(f"Saved top-{n} scan IDs to: {output_txt_path}")

    return top_scans

### Saves top-50 scan IDs with highest IOU values to: /content/drive/MyDrive/data/outputs/exp01_highest_50/top_scans.txt

In [14]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=50,
    strategy='highest',
    output_txt_path='top_n_scans/full_train_dataset/exp01_highest_50_scans.txt'
)

Saved top-50 scan IDs to: top_n_scans/full_train_dataset/exp01_highest_50_scans.txt


In [15]:
# Re-run and Check Consistency
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=50,
    strategy='highest',
    output_txt_path='top_n_scans/full_train_dataset/exp01_highest_50_scans_check.txt'
)

Saved top-50 scan IDs to: top_n_scans/full_train_dataset/exp01_highest_50_scans_check.txt


In [18]:
!diff top_n_scans/full_train_dataset/exp01_highest_50_scans.txt top_n_scans/full_train_dataset/exp01_highest_50_scans_check.txt

In [19]:
# Cross-check a specific scan manually
import csv

scan_id = "r001s023"
total = 0.0
with open("iou_scores.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        if row[0] == 'sample_id':
            continue
        if row[0].startswith(scan_id + "_"):
            total += float(row[1])
print(f"{scan_id} total IoU: {total:.4f}")

r001s023 total IoU: 1.5500


### Saves top-50 scan IDs with lowest IOU values to: /content/drive/MyDrive/data/outputs/exp02_lowest_50/top_scans.txt

In [20]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=50,
    strategy='lowest',
    output_txt_path='top_n_scans/full_train_dataset/exp02_lowest_50_scans.txt'
)

Saved top-50 scan IDs to: top_n_scans/full_train_dataset/exp02_lowest_50_scans.txt


In [21]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=100,
    strategy='highest',
    output_txt_path='top_n_scans/full_train_dataset/exp03_highest_100_scans.txt'
)

Saved top-100 scan IDs to: top_n_scans/full_train_dataset/exp03_highest_100_scans.txt


In [22]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=100,
    strategy='lowest',
    output_txt_path='top_n_scans/full_train_dataset/exp04_lowest_100_scans.txt'
)

Saved top-100 scan IDs to: top_n_scans/full_train_dataset/exp04_lowest_100_scans.txt


In [23]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=200,
    strategy='highest',
    output_txt_path='top_n_scans/full_train_dataset/exp05_highest_200_scans.txt'
)

Saved top-200 scan IDs to: top_n_scans/full_train_dataset/exp05_highest_200_scans.txt


In [24]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=200,
    strategy='lowest',
    output_txt_path='top_n_scans/full_train_dataset/exp06_lowest_200_scans.txt'
)

Saved top-200 scan IDs to: top_n_scans/full_train_dataset/exp06_lowest_200_scans.txt


In [25]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=300,
    strategy='highest',
    output_txt_path='top_n_scans/full_train_dataset/exp07_highest_300_scans.txt'
)

Saved top-300 scan IDs to: top_n_scans/full_train_dataset/exp07_highest_300_scans.txt


In [26]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=300,
    strategy='lowest',
    output_txt_path='top_n_scans/full_train_dataset/exp08_lowest_300_scans.txt'
)

Saved top-300 scan IDs to: top_n_scans/full_train_dataset/exp08_lowest_300_scans.txt


In [14]:
top_scans = select_top_scans_from_csv_and_save(
    iou_csv_path='iou_scores.csv',
    n=524,
    strategy='highest',
    output_txt_path='top_n_scans/full_train_dataset/full_train_dataset_ids.txt'
)

Saved top-524 scan IDs to: top_n_scans/full_train_dataset/full_train_dataset_ids.txt


In [27]:
import csv

scan_ids = set()

with open('iou_scores.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[0] == 'sample_id':
            continue
        sample_id = row[0]
        scan_id = sample_id.split('_')[0]
        scan_ids.add(scan_id)

print(f"Total unique scans: {len(scan_ids)}")

Total unique scans: 524


In [9]:
file_path = "dataset/r001s001_115.npy"

# Load the full array (allow_pickle=True is necessary)
data = np.load(file_path, allow_pickle=True)

# Check how many components there are (should be 6 based on your format)
print(f"Total components in the file: {len(data)}")

# Grab the OPF 3D array (last element)
opf_array = data[-1]

# Display its shape, data type, and a basic statistical summary
print(f"OPF array shape: {opf_array.shape}")
print(f"OPF array dtype: {opf_array.dtype}")
print(f"Min: {opf_array.min():.4f}, Max: {opf_array.max():.4f}, Mean: {opf_array.mean():.4f}")

# Optional: Check if it's 3D and if so, print the last dimension
if opf_array.ndim == 3:
    print(f"OPF array has {opf_array.shape[2]} channels (depth)")

Total components in the file: 6
OPF array shape: (1, 208, 240)
OPF array dtype: float64
Min: 0.0000, Max: 1.0000, Mean: 0.0388
OPF array has 240 channels (depth)


In [10]:
opf_mask = data[5][0]

In [11]:
opf_mask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
opf_array.shape

(1, 208, 240)

In [13]:
img = data[0]

In [14]:
img.shape

(3, 208, 240)

In [15]:
ws_label = data[2]

In [16]:
ws_label.shape

(1, 208, 240)

In [20]:
data[2]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])